In [2]:
import os
import json
import shutil
from sklearn.model_selection import train_test_split

def create_dataset_split(data_dir, output_dir, train_ratio=0.7, val_ratio=0.15, test_ratio=0.15):
    """
    Chia dữ liệu thành train, test, validation và tạo file metadata.jsonl.
    
    Args:
        data_dir (str): Thư mục gốc chứa annotations/ và images/
        output_dir (str): Thư mục đầu ra (dataset_split/)
        train_ratio (float): Tỷ lệ dữ liệu cho tập train
        val_ratio (float): Tỷ lệ dữ liệu cho tập validation
        test_ratio (float): Tỷ lệ dữ liệu cho tập test
    """
    # Đảm bảo tỷ lệ hợp lệ
    assert abs(train_ratio + val_ratio + test_ratio - 1.0) < 1e-5, "Tổng tỷ lệ phải bằng 1.0"

    # Đường dẫn đến thư mục annotations và images
    annotations_dir = os.path.join(data_dir, "annotations")
    images_dir = os.path.join(data_dir, "images")
    
    # Tạo danh sách các file ảnh và file JSON
    image_files = [f for f in os.listdir(images_dir) if f.endswith((".jpg", ".png"))]
    json_files = [f for f in os.listdir(annotations_dir) if f.endswith(".json")]
    
    # Ghép các file ảnh và JSON dựa trên tên
    paired_files = []
    for img_file in image_files:
        # Tìm file JSON tương ứng
        json_file = img_file.rsplit(".", 1)[0] + ".json"
        if json_file in json_files:
            paired_files.append((img_file, json_file))
        else:
            print(f"Cảnh báo: Không tìm thấy file JSON cho ảnh {img_file}, bỏ qua.")
    
    # Chia dữ liệu thành train, validation, test
    train_pairs, temp_pairs = train_test_split(
        paired_files,
        train_size=train_ratio,
        random_state=2022  # Đảm bảo lặp lại được
    )
    val_ratio_adjusted = val_ratio / (val_ratio + test_ratio)
    val_pairs, test_pairs = train_test_split(
        temp_pairs,
        train_size=val_ratio_adjusted,
        random_state=2022
    )
    
    # Tạo thư mục đầu ra
    splits = {
        "train": train_pairs,
        "validation": val_pairs,
        "test": test_pairs
    }
    
    for split, pairs in splits.items():
        split_dir = os.path.join(output_dir, split)
        os.makedirs(split_dir, exist_ok=True)
        
        # Tạo file metadata.jsonl
        metadata_path = os.path.join(split_dir, "metadata.jsonl")
        with open(metadata_path, "w", encoding="utf-8") as f:
            for img_file, json_file in pairs:
                # Đọc file JSON
                with open(os.path.join(annotations_dir, json_file), "r", encoding="utf-8") as jf:
                    annotation = json.load(jf)
                
                # Tạo gt_parse (định dạng Document Parsing)
                gt_parse = {}
                for key, value in annotation.items():
                    gt_parse[key] = value  # Điều chỉnh tùy theo cấu trúc annotation của bạn
                
                # Tạo dòng metadata
                metadata = {
                    "file_name": img_file,
                    "ground_truth": json.dumps({"gt_parse": gt_parse}, ensure_ascii=False)
                }
                f.write(json.dumps(metadata, ensure_ascii=False) + "\n")
                
                # Sao chép file ảnh vào thư mục split
                src_img_path = os.path.join(images_dir, img_file)
                dst_img_path = os.path.join(split_dir, img_file)
                shutil.copy2(src_img_path, dst_img_path)
        
        print(f"Đã tạo split {split} với {len(pairs)} mẫu tại {split_dir}")

if __name__ == "__main__":
    data_dir = "data"
    output_dir = "dataset"
    create_dataset_split(data_dir, output_dir, train_ratio=0.7, val_ratio=0.15, test_ratio=0.15)

Đã tạo split train với 104 mẫu tại dataset\train
Đã tạo split validation với 22 mẫu tại dataset\validation
Đã tạo split test với 23 mẫu tại dataset\test
